# Subsequent Memory Effect

### Imports

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


### Settings

In [2]:
# Paths
PROJECT_PATH = 'C:/Users/micha/projects/oscillation_vs_exponent/'


### Load data

In [3]:
# load aggregted spectral parameterization results
data = pd.read_csv(PROJECT_PATH + 'data/results/' + 'df_ols_allchans.csv')
# data = pd.read_csv(PROJECT_PATH + 'data/results/' + 'df_ols.csv')
print(data.shape)
print(len(data)/4)
data.head()

(2780, 20)
695.0


,Unnamed: 0,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,exp_pre,exp_post,exp_diff,alpha_pre,alpha_post,alpha_diff,peak_present,f_rot,alpha_bp_diff,material,memory,ap_mode
0,0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,3.542673,3.504914,-0.037759,0.213201,0.123599,-0.089602,3.0,10.978558,-1536.029206,words,hit,knee
1,1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,3.052480,2.834013,-0.218467,0.210785,0.225154,0.014369,3.0,83.836257,-212.749318,words,hit,knee
2,2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,2.774045,2.725544,-0.048501,0.276397,NaN,NaN,1.0,89.824561,-2992.875343,words,hit,knee
3,3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,2.940543,2.586441,-0.354102,NaN,0.213641,NaN,2.0,59.883041,-1776.054670,words,hit,knee
4,4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,2.872949,2.557193,-0.315757,0.227912,0.182313,-0.045599,3.0,65.871345,-4302.884091,words,hit,knee


### Run 2x2 Anova

In [4]:
# run 2-way anova (memory x material)

# loop over variables of interest
results = {}
for var in ['exp', 'alpha', 'alpha_bp']:
    # Convert NaNs to 0s
    df = data.copy()
    df.loc[np.isnan(df[f'{var}_diff']), f'{var}_diff'] = 0

    # repeated measures anova
    results_i = sm.stats.AnovaRM(df, f"{var}_diff", 'unique_id', within=['memory', 'material']).fit()

    # aggregate
    results[var] = results_i

    # show results
    print(f"\n\n===================  {var}  ===================\n")
    print(results_i)



===================  exp  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.9523 1.0000 694.0000 0.3295
material         4.0674 1.0000 694.0000 0.0441
memory:material  0.0757 1.0000 694.0000 0.7833



===================  alpha  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.0008 1.0000 694.0000 0.9781
material         0.3503 1.0000 694.0000 0.5541
memory:material  0.3869 1.0000 694.0000 0.5342



===================  alpha_bp  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.8071 1.0000 694.0000 0.3693
material         4.4911 1.0000 694.0000 0.0344
memory:material  0.0000 1.0000 694.0000 0.9956



Original manuscript applied 2x2 at the level of individual electrodes. This will require further modification of the pipeline